This code runs Deep Learning on nutau and nue samples created by Dasha. 

This code runs in a python 3.7 conda environment constructed as follows:
- conda create --name tf-gpu tensorflow-gpu keras scikit-learn matplotlib ipykernel nb_conda_kernels [Pillow]
- conda activate tf-gpu
(Pillow is for image manipulation for making heat maps, but I haven't got it to work yet.  Can remove from environment.)

To run with multiple GPUs on CyberLAMP you must specify the "nodes" and "gpus" qualifiers in the same chunk.  For example:
- qsub -I -A cyberlamp -l qos=cl_higpu -l nodes=1:ppn=1:gpus=4:shared -l mem=24gb -l walltime=4:00:00

In [22]:
import os

# Set which GPU to use.  This probably needs to be done before any other CUDA vars get defined.
# Use the command "nvidia-smi" to get association of a particular GPU with a particular number.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

In [23]:
from __future__ import division
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [24]:
c =0.299792458
n =1.3195
v=c/n 
print(v)

0.2272015596816976


In [25]:
info_d = []
info_s = []
data_d = []
data_s = []

for i in range(0,10):
    name_dd = "/home/dup193/work/double_pulse/data/Tau05to15PeV_00{0}_data.npy".format(i)
    name_di = "/home/dup193/work/double_pulse/data/Tau05to15PeV_00{0}_info.pkl".format(i)
    name_sd = "/home/dup193/work/double_pulse/data/Electron05to15PeV_00{0}_data.npy".format(i)
    name_si = "/home/dup193/work/double_pulse/data/Electron05to15PeV_00{0}_info.pkl".format(i)
    info_d_temp = pickle.load(open(name_di, "rb"))
    info_s_temp = pickle.load(open(name_si, "rb"))
    data_d_temp = np.load(name_dd ,allow_pickle=True,encoding='bytes')
    data_s_temp = np.load(name_sd ,allow_pickle=True,encoding='bytes')
    info_d = info_d + info_d_temp
    info_s = info_s + info_s_temp
    data_d.append(data_d_temp)    
    data_s.append(data_s_temp)
data_d = np.vstack(data_d)
data_s = np.vstack(data_s)
info_d = np.array(info_d)
info_s = np.array(info_s)

In [26]:
energy_l_d = []
energy_nu_d = []
charge_d = []
charge_st_d = []
for i in info_d:
    energy_l_d.append(i['tau_energy'])
    energy_nu_d.append(i['nu_energy'])
    charge_d.append(i['qtotal'])
    charge_st_d.append(i['strings']['charge'])

energy_l_d = np.array(energy_l_d)
energy_nu_d = np.array(energy_nu_d)
charge_d = np.array(charge_d)
charge_st_d = np.array(charge_st_d)

energy_l_s = []
energy_nu_s = []
charge_s = []
charge_st_s = []
for i in info_s:
    energy_l_s.append(i['tau_energy'])
    energy_nu_s.append(i['nu_energy'])
    charge_s.append(i['qtotal'])
    charge_st_s.append(i['strings']['charge'])

energy_l_s = np.array(energy_l_s)
energy_nu_s = np.array(energy_nu_s)
charge_s = np.array(charge_s)
charge_st_s = np.array(charge_st_s)
 
print(len(energy_l_d),len(energy_nu_d),len(charge_d),len(charge_st_d))
print(len(energy_l_s),len(energy_nu_s),len(charge_s),len(charge_st_s))

10062 10062 10062 10062
24019 24019 24019 24019


In [27]:
def Plot(arr1,arr2, bs =100, r = [0,100], lab = "Charge"):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    weights1 = np.ones_like(arr1)/float(len(arr1))
    weights2 = np.ones_like(arr2)/float(len(arr2))
    #ax.set_yscale(scale)
    #ax.set_title(name, fontsize = 14)
    ax.text(0.5,0.95,'#Events single '+str(len(arr1)), transform=ax.transAxes, color = "black",fontsize=8)
    ax.text(0.5,0.90,'#Events double '+str(len(arr2)), transform=ax.transAxes, color = "black",fontsize=8)
    ax.set_xlabel(lab, fontsize = 14)                                                              
    ax.set_ylabel("FractionEvents", fontsize = 14)                                                   
    ax.hist(arr1, bins =bs, weights = weights1, range = r, histtype = 'step',edgecolor ='r', fill= False, label = 'Single')
    ax.hist(arr2, bins =bs, weights = weights2, range = r, histtype = 'step',edgecolor ='b', fill= False, label = 'Double')


In [28]:
#Plot(energy_l_s/1000000,energy_l_d/1000000, bs =50, r = [0.4,1.6], lab = "energy of lepton, GeV")

In [29]:
#Plot(energy_nu_s/1000000,energy_nu_d/1000000, bs =50, r = [0.4,3.6], lab = "energy of neutrino, GeV")

In [30]:
#Plot(charge_s,charge_d,bs = 50, r= [0,200000], lab = "total charge, Pe")

In [31]:
#Plot(charge_st_s,charge_st_d,bs = 50, r= [0,200000], lab = "string charge, Pe")

In [32]:
print(info_d.shape, data_d.shape)
print(info_s.shape, data_s.shape)
print(type(data_d_temp[0][0][0]))

(10062,) (10062, 300, 60)
(24019,) (24019, 300, 60)
<class 'numpy.float64'>


In [33]:
print(np.amax(data_d))
print(np.amin(data_d))
print(np.amax(data_s))
print(np.amin(data_s))

6.405907616717559e-09
-5.962297593924894e-11
6.883592770365534e-09
-6.128851422642174e-11


In [34]:
from keras.utils import to_categorical
label_d = [[0,1]]*len(data_d)
label_s = [[1,0]]*len(data_s)
label_d = np.array(label_d)
label_s = np.array(label_s)
print(label_d.shape)
print(label_s.shape)
print(label_d[0])
print(label_s[0])

(10062, 2)
(24019, 2)
[0 1]
[1 0]


In [35]:
from sklearn.utils import shuffle

In [36]:
data = np.concatenate((data_d, data_s), axis = 0) 
label = np.concatenate((label_d, label_s), axis = 0) 
data, label = shuffle(data, label, random_state =12)

train_data = data[:24000]
train_label = label[:24000]
train_data = train_data.reshape((len(train_data),300,60,1))
train_data = train_data.astype('float32')/10**-8
mean = np.mean(train_data)
std = np.std(train_data)
print(mean,std)
train_data = train_data - mean
train_data = train_data/std

valid_data = data[24000:28000]
valid_label = label[24000:28000]
valid_data = valid_data.reshape((len(valid_data),300,60,1))
valid_data = valid_data.astype('float32')/10**-8
valid_data = valid_data - mean
valid_data = valid_data/std

test_data = data[28000:]
test_label = label[28000:]
test_data = test_data.reshape((len(test_data),300,60,1))
test_data = test_data.astype('float32')/10**-8
test_data = test_data - mean
test_data = test_data/std


0.0026542188 0.022200854


In [37]:
print(np.amax(train_data))
print(np.amin(train_data))

30.886427
-0.3950856


In [38]:
from keras import layers
from keras import models

In [39]:
#for im in data_d[:100]:
#    print(im.shape)
#    fig = plt.figure(figsize=(12, 12))
#    ax = fig.add_subplot(111)
#    ax.imshow(im, interpolation='nearest', aspect='auto', cmap= 'gray')
    #ax.plot(im[14,:])
    #print(im[:,46])

In [40]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300, 60, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 298, 58, 32)       320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 29, 32)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 147, 27, 64)       18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 13, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 71, 11, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 5, 128)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 33, 3, 128)        147584    
__________

In [41]:
from keras import optimizers
import tensorflow as tf

run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

sgd = optimizers.SGD(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
#              options = run_opts)

In [42]:
history = model.fit(train_data,train_label, epochs=5, validation_data=(valid_data,valid_label), batch_size =128)

Train on 24000 samples, validate on 4000 samples
Epoch 1/5
24000/24000 [==============================] - 8s 353us/step - loss: 0.6118 - acc: 0.7076 - val_loss: 0.6062 - val_acc: 0.7053
Epoch 2/5
24000/24000 [==============================] - 8s 351us/step - loss: 0.6015 - acc: 0.7076 - val_loss: 0.6040 - val_acc: 0.7053
Epoch 3/5
24000/24000 [==============================] - 9s 379us/step - loss: 0.5988 - acc: 0.7077 - val_loss: 0.6015 - val_acc: 0.7053
Epoch 4/5
24000/24000 [==============================] - 9s 367us/step - loss: 0.5981 - acc: 0.7078 - val_loss: 0.6015 - val_acc: 0.7050
Epoch 5/5
24000/24000 [==============================] - 9s 378us/step - loss: 0.5968 - acc: 0.7080 - val_loss: 0.5983 - val_acc: 0.7053


In [55]:
import json
history_dict = history.history # Get the dictionary containing each metric and the loss for each epoch
json.dump(history_dict, open('/data/dfc13/DoublePulse/Models/DP_model_10g.json', 'w')) # Save it under the form of a json file
#
model.save('/data/dfc13/DoublePulse/Models/DP_model_10g.h5') # Save the model
#
ResultsFile = open('/data/dfc13/DoublePulse/Models/DP_model_10g.txt','w') # File for saving the results of the fit.
#
# 8: multiple conv2d layers a la textbook (32 (3,3) then 64, 128, 128, dropout 0.3, dense 256(relu), 64(relu), 2(softmax)
# 8b: same as 8 but with 50 trials
# 8b[not c didn't update the line above]: same as 8b but with dropout 0.5.
# 8c: changed SGC "lr" from 0.01->0.001.  This smoothed things out a lot.  Sort of.  But had 0% accuracy for nutaus.
# 8d: changed SGC "lr" from 0.001->0.005.
# 8e: decreased size of training set to 24k, increased size of validation set to 4k (24k:28k) and increased test set (28k:)
# 8f: changed SGC "lr" from 0.005->0.01.  200 epochs. 97%/37%.
# 8g: lr=0.02, 50 epochs: 98%/29%.

# 9a: changed dense 256 -> 512.  50 epochs. 100%/22%
# 9b: same as above, 200 epochs. 97%/39%.
# 9c: same as above, lr = 0.01.  200 epochs. 97%/36%.
# 9d: same as above, 500 epochs batch_size = 64. 96%/40%.
# 10a: Testing selection of particular GPU.  20 epochs, batch_size = 128.
# 10b: Testing parallelization.  50 epochs, batch_size = 128, one GPU. 98%/22%.
# 10g: 

In [56]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

#plt.plot(epochs, acc, 'bo', label='Training acc')
#plt.plot(epochs, val_acc, 'b', label='Validation acc')
#plt.title('Training and validation accuracy')
#plt.legend()

#plt.figure()

#plt.plot(epochs, loss, 'bo', label='Training loss')
#plt.plot(epochs, val_loss, 'b', label='Validation loss')
#plt.title('Training and validation loss')
#plt.legend()

#plt.show()

In [57]:
result = model.evaluate(test_data,test_label)
print(result)

6081/6081 [==============================] - 1s 241us/step
[0.608180747205559, 0.693471468508469]


In [58]:
from sklearn.metrics import classification_report, confusion_matrix

In [59]:
test_pred = model.predict(test_data)
matrix = confusion_matrix(test_label.argmax(axis=1), test_pred.argmax(axis=1))
report = classification_report(test_label.argmax(axis=1), test_pred.argmax(axis=1))

In [60]:
#print("True Positive",tp)
#print("True Negative",tn)
#print("False Positive",fp)
#print("False Negative",fn)
print(matrix)
print(report)

ResultsFile.write(repr(matrix))
ResultsFile.write('\n')
ResultsFile.write(report)
ResultsFile.close()

[[4216    0]
 [1864    1]]
              precision    recall  f1-score   support

           0       0.69      1.00      0.82      4216
           1       1.00      0.00      0.00      1865

   micro avg       0.69      0.69      0.69      6081
   macro avg       0.85      0.50      0.41      6081
weighted avg       0.79      0.69      0.57      6081

